In [1]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

station = 'LVY'
year = '2023'
channels = ['BHN','BHE','BHZ']

for channel in channels:
    file_path = f'/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}/{channel}/{station}_{channel}_{year}-01-09.mseed'
    stream = read(file_path)
    print(stream)

1 Trace(s) in Stream:
CI.LVY..BHN | 2023-01-09T00:00:00.024999Z - 2023-01-09T23:59:59.974999Z | 40.0 Hz, 3455999 samples
1 Trace(s) in Stream:
CI.LVY..BHE | 2023-01-09T00:00:00.024999Z - 2023-01-09T23:59:59.974999Z | 40.0 Hz, 3455999 samples
1 Trace(s) in Stream:
CI.LVY..BHZ | 2023-01-09T00:00:00.024999Z - 2023-01-09T23:59:59.974999Z | 40.0 Hz, 3455999 samples


In [2]:
stream[0].stats

         network: CI
         station: LVY
        location: 
         channel: BHZ
       starttime: 2023-01-09T00:00:00.024999Z
         endtime: 2023-01-09T23:59:59.974999Z
   sampling_rate: 40.0
           delta: 0.025
            npts: 3455999
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 15724, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 8050688})

In [ ]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

def format_duration(duration):
    """Convert duration from seconds to hours:minutes format."""
    hours = int(duration // 3600)
    minutes = int((duration % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

def save_processed_trace(trace, original_path):
    """Save the processed trace to a new file with '_processed' appended to the original name."""
    directory, filename = os.path.split(original_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_processed{ext}"
    new_path = os.path.join(directory, new_filename)
    trace.write(new_path, format='MSEED')
    logging.info(f"Saved processed trace to: {new_path}")

def process_waveform(file_path):
    try:
        stream = read(file_path)
        #print(f"Number of traces: {len(stream)}")
        if len(stream) > 1:
            print(stream)
        
        
        stream.merge(method=1, fill_value=0)
        duration = stream[0].stats.endtime - stream[0].stats.starttime
        duration_str = format_duration(duration)
        logging.info(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")
        #print(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")

        # Save the processed trace
        save_processed_trace(stream[0], file_path)

    except Exception as e:
        logging.error(f"Error processing file {stream[0]}: {str(e)}")
        print(f"Error processing file {stream[0]}: {str(e)}")



base_dir = f"/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}"

logging.basicConfig(filename=f"{year}_{station}_trace_preprocessing.log", level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')
# Base directory to start searching


print(f'processing year {year}')
# Collect all mseed files recursively in a list
file_list = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".mseed"):
            file_path = os.path.join(root, file)
            file_list.append(file_path)

# Sort the list of files alphabetically
file_list.sort()
print(f"In total {len(file_list)} files")

# Process each file that has not been processed yet, total 1095 in 2021
for file_path in file_list:
    # Skip already processed files
    if "_processed" in os.path.basename(file_path):
        logging.info(f"Skipping already processed file: {os.path.basename(file_path)}")
        continue
    
    #print(f"Processing file {file_path}")
    logging.info(f"Processing file {os.path.basename(file_path)}")
    process_waveform(file_path)

processing year 2023
In total 1095 files
12 Trace(s) in Stream:
CI.LVY..BHE | 2023-01-03T00:00:00.000000Z - 2023-01-03T13:26:17.400000Z | 40.0 Hz, 1935097 samples
CI.LVY..BHE | 2023-01-03T13:26:22.625000Z - 2023-01-03T13:26:38.650000Z | 40.0 Hz, 642 samples
CI.LVY..BHE | 2023-01-03T13:26:49.275000Z - 2023-01-03T13:27:05.050000Z | 40.0 Hz, 632 samples
CI.LVY..BHE | 2023-01-03T13:27:10.225000Z - 2023-01-03T13:27:26.200000Z | 40.0 Hz, 640 samples
CI.LVY..BHE | 2023-01-03T13:27:31.575000Z - 2023-01-03T13:28:39.825000Z | 40.0 Hz, 2731 samples
CI.LVY..BHE | 2023-01-03T13:30:56.100000Z - 2023-01-03T13:31:07.225000Z | 40.0 Hz, 446 samples
CI.LVY..BHE | 2023-01-03T13:31:12.675000Z - 2023-01-03T13:31:17.975000Z | 40.0 Hz, 213 samples
CI.LVY..BHE | 2023-01-03T13:31:28.325000Z - 2023-01-03T13:31:43.850000Z | 40.0 Hz, 622 samples
CI.LVY..BHE | 2023-01-03T13:31:54.325000Z - 2023-01-03T13:32:04.900000Z | 40.0 Hz, 424 samples
CI.LVY..BHE | 2023-01-03T13:32:10.125000Z - 2023-01-03T13:32:32.025000Z | 40

In [ ]:

channel = 'BHN' #'BHE','BHZ'


file_path = f'/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}/{channel}/{station}_{channel}_{year}-07-23.mseed'
stream = read(file_path)
print(stream)